In [1]:
from modules.tweet_data import read_raw_data
from modules.topics import TopicSeries, display_components

import datetime as dt
import pandas as pd
import spacy
import re
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

import matplotlib.pyplot as plt

In [2]:
tweet_df = read_raw_data()
es = pd.read_csv('data/ES.csv')

Reading data


In [3]:
es = es[es.Time == '15:00'][['Date','Time','Close']]
es.Date = pd.to_datetime(es.Date +' ' +'15:45')
es.drop('Time',axis=1,inplace=True)
es.set_index('Date', inplace=True)

In [4]:
date_range = es['2011-12-30':'2020-5-31'].index

In [5]:
batch = date_range[-100:]
#date_range[-1200:-799]
#batch

In [ ]:
ts=TopicSeries()
for i in range(len(batch)-1):
    str_date = str(batch[i+1].date())
    print(str_date)
    ts.calculate_nmf(str_date, 
                     tweet_df[batch[i]:(batch[i+1]-dt.timedelta(seconds=1))].tweet)

2020-01-08
2020-01-09
2020-01-10
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-21
2020-01-22
2020-01-23
2020-01-24
2020-01-27
2020-01-28
2020-01-29
2020-01-30
2020-01-31
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-23
2020-03-24


C:\Users\PC\Anaconda3\envs\tweet-sentiment\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


2020-03-25
2020-03-26
2020-03-27
2020-03-30


In [25]:
pickle.dump( ts, open( "NMF-2018-10-19-2020-05-29.p", "wb" ) )